In [1]:
from coffea import hist
import math

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
import numpy as np
from itertools import chain

plt.style.use(hep.style.CMS)

import awkward as ak

from matplotlib import colors
POPTS={'norm':colors.LogNorm()}

In [2]:
from data import getData, repackage
ldmx_dict = getData(chunks=True, fnames="/Users/chloeg/Desktop/Work/Fermilab2021/HistData/kaon_pn_4GeV_Jul21_ntuple/*.root")




In [3]:
def extend_array(arr, new_attr, new_attr_name):
    members={n:arr[n] for n in arr.fields}
    members[new_attr_name] = new_attr
    return ak.zip(members)

def add_angle(arr,br=['px','py','pz','e'],name="theta"):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    arr = extend_array(arr, part.theta, name)
    return arr

def get_vector(arr):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    return part

def flat(x,axis=None): # for now must cast while waiting for coffea to catch up
    try:
        return ak.to_numpy(ak.flatten(x,axis=axis)) 
    except:
        return x

In [4]:
hists = {}

hists["hist_pdgid1"] = hist.Hist("Sim Particle",
                                hist.Cat("Ptype", "Type of Particle"),
                                hist.Bin("e", r"PDG ID", 40, -400, 4000)
                            )
hists["hist_pdgid2"] = hist.Hist("Sim Particle",
                                hist.Cat("Ptype", "Type of Particle"),
                                hist.Bin("e", r"PDG ID", 40, -400, 4000)
                            )
hists["hist_ke1"] = hist.Hist("Sim Particle",
                                hist.Cat("Ptype", "Type of Particle"),
                                hist.Bin("e", r"Kinetic Energy [MeV]", 50, 4, 3500),
                                )
hists["hist_ke2"] = hist.Hist("Sim Particle",
                                hist.Cat("Ptype", "Type of Particle"),
                                hist.Bin("e", r"Kinetic Energy [MeV]", 50, 4, 3500),
                                )

In [5]:
def ProcessChunk(chunk, hists):
    ldmx_events = repackage(chunk)
    
    sim_particle = ldmx_events['Sim_PNParticle']
    sim_particle_n = ldmx_events['n']['Sim_PNParticle']
    
    masks_id = {'Kplus': 321,
                'Kshort': 310,
                'Klong': 130, 
                'Kminus':-321,
                'Electron': 11,
                'Positron': -11,
                'Pion': 211,
                'Pi-': -211,
                'Proton':2212,
                'Neutron':2112,
                'Sigma': 3212,
                'Lambda': 3122,
               }
    masses =   {'Kplus': 493.677,
                'Kshort': 493.677,
                'Klong': 493.677,
                'Kminus': 493.677,
                'Electron': 0.511,
                'Positron': 0.511,
                'Pion': 139.57039,
                'Pi-': 139.57039,
                'Proton': 939.565,
                'Neutron': 939.565,
                'Sigma': 1189.36,
                'Lambda': 1115.6,
                }
    
    kshort_mask = sim_particle.pdgID == 310
    klong_mask  = sim_particle.pdgID == 130
    kplus_mask  = sim_particle.pdgID == 321
    kminus_mask = sim_particle.pdgID == -321
    kaon_mask = kshort_mask | klong_mask | kplus_mask | kminus_mask
    n_kaons  = ak.sum(kaon_mask, axis = 1)
    n_kshort = ak.sum(kshort_mask, axis = 1)
    n_klong  = ak.sum(klong_mask, axis = 1)
    n_kplus  = ak.sum(kplus_mask, axis = 1)
    n_kminus  = ak.sum(kminus_mask, axis = 1)
    print("Number of Kaons:", n_kaons)
    print(sim_particle[kaon_mask].pdgID[n_kaons>2])
    print("Number of Kshort:", n_kshort)
    print("Number of Klong:", n_klong)
    print("Number of Kplus:", n_kplus)
    print("Number of Kminus:", n_kminus)
    print()

In [6]:
nchunk = 0
for chunk in ldmx_dict:
    nchunk += 1
    print('process',nchunk)
    ProcessChunk(chunk, hists)
    

process 1
Number of Kaons: [2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, ... 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2]
[]
Number of Kshort: [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, ... 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 1, 0]
Number of Klong: [1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, ... 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Number of Kplus: [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, ... 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1]
Number of Kminus: [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ... 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]

process 2
Number of Kaons: [1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, ... 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2]
[]
Number of Kshort: [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, ... 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1]
Number of Klong: [0, 1, 1, 0, 2, 0, 0, 1, 0, 0, 0, 0, 1, 1, ... 0, 0, 1, 1, 0, 2, 0, 0, 0, 1, 0, 1, 1]
Number of Kplus: [1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, ... 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
Number of Kminus: [0, 0, 0, 0, 0, 1, 0, 0, 

process 17
Number of Kaons: [1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, ... 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2]
[]
Number of Kshort: [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, ... 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1]
Number of Klong: [0, 0, 1, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, ... 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
Number of Kplus: [1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, ... 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0]
Number of Kminus: [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, ... 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]

process 18
Number of Kaons: [1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, ... 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2]
[]
Number of Kshort: [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, ... 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1]
Number of Klong: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, ... 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0]
Number of Kplus: [0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, ... 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0]
Number of Kminus: [0, 1, 0, 1, 0, 0, 1, 0

process 33
Number of Kaons: [1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, ... 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1]
[]
Number of Kshort: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ... 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0]
Number of Klong: [0, 0, 1, 1, 0, 0, 1, 1, 0, 2, 0, 1, 0, 1, ... 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0]
Number of Kplus: [0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, ... 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1]
Number of Kminus: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ... 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

process 34
Number of Kaons: [1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, ... 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2]
[[130, 310, 310]]
Number of Kshort: [0, 1, 0, 0, 0, 2, 0, 0, 1, 0, 0, 1, 1, 0, ... 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0]
Number of Klong: [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ... 1, 1, 0, 2, 0, 1, 1, 1, 0, 0, 1, 1, 0]
Number of Kplus: [0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, ... 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1]
Number of Kminus: [0, 1, 0

process 49
Number of Kaons: [2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, ... 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1]
[]
Number of Kshort: [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, ... 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1]
Number of Klong: [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ... 0, 0, 0, 2, 1, 0, 0, 1, 0, 1, 0, 1, 0]
Number of Kplus: [1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, ... 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
Number of Kminus: [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ... 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0]

process 50
Number of Kaons: [1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, ... 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1]
[]
Number of Kshort: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ... 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0]
Number of Klong: [0, 1, 0, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 0, ... 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Number of Kplus: [0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, ... 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1]
Number of Kminus: [0, 0, 0, 0, 0, 0, 0, 0

process 65
Number of Kaons: [2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, ... 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1]
[]
Number of Kshort: [1, 1, 0, 1, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0, ... 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0]
Number of Klong: [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, ... 1, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 1]
Number of Kplus: [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, ... 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
Number of Kminus: [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]

process 66
Number of Kaons: [2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2]
[]
Number of Kshort: [1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ... 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]
Number of Klong: [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, ... 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0]
Number of Kplus: [0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, ... 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1]
Number of Kminus: [1, 1, 0, 0, 1, 0, 0, 0

...

process 305
Number of Kaons: [1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, ... 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 1]
[[130, -321, 321]]
Number of Kshort: [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, ... 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0]
Number of Klong: [1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, ... 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Number of Kplus: [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, ... 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1]
Number of Kminus: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0]

process 306
Number of Kaons: [1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, ... 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2]
[]
Number of Kshort: [0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, ... 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0]
Number of Klong: [0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, ... 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
Number of Kplus: [1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, ... 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
Number of Kminus: [0, 0

process 321
Number of Kaons: [1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, ... 1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1]
[]
Number of Kshort: [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ... 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Number of Klong: [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ... 0, 1, 0, 1, 2, 0, 0, 1, 0, 0, 1, 0, 1]
Number of Kplus: [1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, ... 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0]
Number of Kminus: [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, ... 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]

process 322
Number of Kaons: [1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, ... 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1]
[[130, -321, 310]]
Number of Kshort: [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, ... 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
Number of Klong: [1, 0, 0, 0, 2, 1, 2, 0, 2, 1, 0, 0, 0, 0, ... 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Number of Kplus: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ... 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0]
Number of Kminus: [0, 1

process 337
Number of Kaons: [1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, ... 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1]
[]
Number of Kshort: [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Number of Klong: [0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, ... 1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0]
Number of Kplus: [1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, ... 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1]
Number of Kminus: [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ... 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

process 338
Number of Kaons: [1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, ... 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1]
[]
Number of Kshort: [0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, ... 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1]
Number of Klong: [0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, ... 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
Number of Kplus: [1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, ... 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Number of Kminus: [0, 0, 0, 0, 1, 0, 0,

process 353
Number of Kaons: [1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, ... 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1]
[]
Number of Kshort: [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ... 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0]
Number of Klong: [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ... 0, 0, 2, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0]
Number of Kplus: [0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1]
Number of Kminus: [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, ... 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

process 354
Number of Kaons: [1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, ... 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1]
[]
Number of Kshort: [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, ... 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 1]
Number of Klong: [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, ... 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0]
Number of Kplus: [0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, ... 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0]
Number of Kminus: [0, 0, 0, 0, 0, 0, 1,

process 369
Number of Kaons: [2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, ... 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2]
[]
Number of Kshort: [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ... 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1]
Number of Klong: [1, 1, 0, 2, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, ... 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0]
Number of Kplus: [0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, ... 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0]
Number of Kminus: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ... 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1]

process 370
Number of Kaons: [2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, ... 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2]
[]
Number of Kshort: [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, ... 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 1]
Number of Klong: [1, 0, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, ... 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]
Number of Kplus: [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, ... 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0]
Number of Kminus: [0, 0, 0, 0, 1, 0, 0,